In [13]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time
from time import time as timer

import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

In [14]:
option = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.2,
    'gpu': 0.85
}

tfnet = TFNet(option)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.056246280670166016s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep

## With Video Resizing

In [18]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('temp.avi',fourcc, 30.0, (960, 540))     # resize image half

start = timer()

count = 0
while(True):
    ret, frame = capture.read() 

    if ret == True:
        
        count += 1
        
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
                    
        results = tfnet.return_predict(frame)
        
        #print('Frame Number {}: Found {} boxes for {}'.format(count - 1, len(results), 'img'))
        
        for result in results:
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            label = result['label']
            confidence = result['confidence']
            confidence = round(confidence, 2)
            frame = cv2.rectangle(frame, tl, br, (255, 255, 0), 7)
            #frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 255), 2)
            frame = cv2.putText(frame, label + ',' + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 255), 2)
            
            Label_Confidence = '{}, {:.2f}'.format(result['label'], result['confidence'])
            TLXY = '{}, {}'.format(result['topleft']['x'], result['topleft']['y'])
            BRXY = '{}, {}'.format(result['bottomright']['x'] , result['bottomright']['y'])
            
            print("{}, {}, {}, {}".format(count - 1 , Label_Confidence, TLXY, BRXY))
        
        out.write(frame)
        cv2.imshow('frame', frame)

        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break

#print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

#print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

0, car, 3, 0.64, 530, 50, 576, 81
0, truck, 8, 0.43, 445, 64, 516, 129
0, car, 3, 0.24, 684, 101, 760, 150
0, car, 3, 0.42, 690, 133, 747, 182
0, car, 3, 0.45, 58, 186, 158, 253
0, car, 3, 0.49, 702, 176, 774, 255
0, truck, 8, 0.40, 238, 70, 334, 122
0, truck, 8, 0.25, 150, 100, 231, 158
1, car, 3, 0.21, 564, 7, 590, 22
1, car, 3, 0.20, 513, 29, 544, 45
1, car, 3, 0.50, 531, 51, 575, 81
1, car, 3, 0.51, 690, 133, 746, 183
1, car, 3, 0.46, 57, 186, 158, 254
1, car, 3, 0.53, 701, 177, 775, 254
1, truck, 8, 0.50, 442, 65, 515, 132
1, truck, 8, 0.48, 239, 68, 333, 122
1, truck, 8, 0.27, 147, 100, 233, 159
1, traffic light, 10, 0.21, 532, 40, 585, 84
2, car, 3, 0.21, 511, 31, 543, 47
2, car, 3, 0.61, 529, 53, 576, 82
2, car, 3, 0.24, 684, 101, 758, 152
2, car, 3, 0.51, 690, 132, 747, 185
2, car, 3, 0.49, 701, 176, 775, 253
2, car, 3, 0.50, 66, 198, 163, 266
2, truck, 8, 0.47, 238, 70, 331, 122
2, truck, 8, 0.59, 440, 67, 513, 135
2, truck, 8, 0.29, 146, 101, 233, 162
2, traffic light, 10, 0

28, car, 3, 0.34, 515, 28, 547, 43
28, car, 3, 0.42, 492, 68, 561, 121
28, car, 3, 0.52, 689, 132, 748, 183
28, car, 3, 0.51, 55, 184, 161, 253
28, car, 3, 0.49, 700, 180, 775, 252
28, car, 3, 0.25, 0, 215, 57, 320
28, bus, 6, 0.37, 355, 91, 460, 205
28, truck, 8, 0.37, 241, 66, 331, 121
28, truck, 8, 0.49, 361, 94, 462, 205
29, car, 3, 0.32, 517, 27, 548, 42
29, car, 3, 0.44, 490, 68, 558, 122
29, car, 3, 0.20, 683, 104, 756, 152
29, car, 3, 0.52, 689, 131, 748, 183
29, car, 3, 0.51, 55, 184, 161, 253
29, car, 3, 0.48, 700, 180, 774, 252
29, car, 3, 0.25, 0, 215, 56, 320
29, truck, 8, 0.39, 347, 105, 455, 207
29, bus, 6, 0.52, 354, 93, 459, 206
29, truck, 8, 0.39, 243, 66, 331, 121
30, car, 3, 0.31, 519, 27, 549, 42
30, car, 3, 0.39, 492, 68, 555, 126
30, car, 3, 0.21, 683, 103, 757, 152
30, car, 3, 0.50, 688, 131, 749, 183
30, car, 3, 0.51, 56, 184, 161, 253
30, car, 3, 0.47, 699, 179, 775, 252
30, car, 3, 0.25, 0, 215, 56, 320
30, truck, 8, 0.43, 347, 99, 455, 207
30, bus, 6, 0.45, 

58, truck, 8, 0.52, 421, 97, 515, 212
58, car, 3, 0.54, 690, 133, 749, 183
58, car, 3, 0.61, 53, 181, 157, 255
58, car, 3, 0.39, 702, 180, 776, 253
58, car, 3, 0.25, 0, 216, 58, 319
58, bus, 6, 0.56, 155, 182, 337, 398
58, truck, 8, 0.39, 242, 65, 336, 122
59, car, 3, 0.22, 520, 28, 547, 43
59, truck, 8, 0.56, 413, 97, 510, 216
59, car, 3, 0.51, 690, 132, 750, 182
59, car, 3, 0.59, 52, 182, 159, 255
59, car, 3, 0.39, 703, 179, 775, 253
59, car, 3, 0.25, 0, 214, 57, 322
59, car, 3, 0.20, 144, 163, 328, 398
59, bus, 6, 0.65, 143, 182, 330, 405
59, truck, 8, 0.35, 244, 66, 335, 121
60, car, 3, 0.21, 520, 28, 549, 42
60, car, 3, 0.51, 688, 132, 749, 183
60, car, 3, 0.60, 51, 181, 160, 254
60, car, 3, 0.41, 702, 182, 776, 251
60, car, 3, 0.27, 0, 215, 60, 320
60, bus, 6, 0.48, 135, 190, 324, 405
60, truck, 8, 0.29, 243, 67, 335, 122
60, truck, 8, 0.57, 408, 97, 509, 220
61, car, 3, 0.21, 520, 28, 549, 42
61, car, 3, 0.51, 689, 133, 749, 183
61, car, 3, 0.62, 52, 182, 158, 254
61, car, 3, 0.

88, car, 3, 0.21, 519, 27, 546, 42
88, car, 3, 0.21, 683, 102, 759, 155
88, car, 3, 0.52, 689, 132, 751, 183
88, car, 3, 0.51, 54, 184, 159, 252
88, car, 3, 0.45, 702, 181, 778, 252
88, car, 3, 0.23, 0, 215, 57, 321
88, car, 3, 0.41, 242, 186, 443, 421
88, truck, 8, 0.39, 244, 65, 334, 122
89, person, 1, 0.43, 468, 502, 501, 535
89, person, 1, 0.30, 498, 504, 527, 532
89, car, 3, 0.20, 519, 27, 546, 42
89, car, 3, 0.20, 684, 102, 759, 155
89, car, 3, 0.52, 689, 132, 751, 183
89, car, 3, 0.51, 54, 184, 159, 252
89, car, 3, 0.45, 702, 181, 778, 252
89, car, 3, 0.22, 0, 215, 56, 321
89, car, 3, 0.30, 242, 193, 438, 421
89, truck, 8, 0.38, 243, 65, 335, 122
89, cell phone, 68, 0.33, 238, 210, 440, 425
90, person, 1, 0.27, 463, 470, 533, 535
90, car, 3, 0.21, 683, 102, 759, 155
90, car, 3, 0.53, 689, 132, 751, 183
90, car, 3, 0.52, 54, 184, 160, 252
90, car, 3, 0.48, 703, 181, 778, 252
90, car, 3, 0.22, 0, 215, 56, 321
90, truck, 8, 0.38, 244, 65, 334, 123
90, cell phone, 68, 0.51, 224, 202

118, person, 1, 0.39, 584, 139, 603, 180
118, person, 1, 0.21, 579, 160, 598, 207
118, person, 1, 0.55, 578, 135, 605, 222
118, car, 3, 0.22, 519, 29, 547, 43
118, car, 3, 0.51, 689, 133, 748, 181
118, car, 3, 0.55, 54, 183, 161, 256
118, car, 3, 0.49, 703, 180, 778, 252
118, car, 3, 0.28, 0, 213, 57, 321
118, truck, 8, 0.28, 240, 66, 333, 122
118, traffic light, 10, 0.31, 5, 439, 223, 539
119, person, 1, 0.61, 581, 124, 605, 204
119, person, 1, 0.22, 578, 160, 599, 205
119, car, 3, 0.23, 519, 29, 547, 43
119, car, 3, 0.52, 689, 133, 749, 181
119, car, 3, 0.55, 54, 183, 161, 256
119, car, 3, 0.50, 702, 180, 778, 251
119, car, 3, 0.29, 0, 213, 57, 321
119, truck, 8, 0.29, 240, 66, 333, 122
119, cell phone, 68, 0.32, 0, 449, 214, 539
120, person, 1, 0.65, 583, 119, 608, 206
120, car, 3, 0.22, 519, 30, 546, 44
120, car, 3, 0.50, 688, 133, 750, 182
120, car, 3, 0.56, 52, 180, 163, 257
120, car, 3, 0.44, 700, 179, 778, 254
120, truck, 8, 0.44, 238, 64, 335, 122
120, suitcase, 29, 0.32, 3, 4

154, person, 1, 0.28, 600, 57, 616, 81
154, car, 3, 0.54, 689, 132, 750, 183
154, car, 3, 0.54, 56, 186, 160, 254
154, car, 3, 0.49, 703, 182, 778, 251
154, truck, 8, 0.43, 239, 64, 335, 123
155, person, 1, 0.24, 600, 56, 616, 81
155, car, 3, 0.20, 518, 27, 546, 42
155, car, 3, 0.53, 689, 132, 749, 183
155, car, 3, 0.54, 56, 186, 161, 254
155, car, 3, 0.48, 703, 181, 778, 251
155, truck, 8, 0.43, 239, 64, 335, 122
156, car, 3, 0.53, 690, 131, 751, 183
156, car, 3, 0.54, 57, 183, 160, 255
156, car, 3, 0.51, 702, 181, 778, 251
156, truck, 8, 0.36, 243, 65, 335, 122
157, car, 3, 0.53, 690, 131, 751, 183
157, car, 3, 0.54, 57, 184, 160, 254
157, car, 3, 0.51, 702, 181, 779, 251
157, truck, 8, 0.37, 243, 65, 335, 121
157, parking meter, 13, 0.24, 600, 55, 618, 80
158, car, 3, 0.51, 690, 131, 751, 183
158, car, 3, 0.53, 57, 184, 159, 255
158, car, 3, 0.49, 701, 181, 778, 251
158, truck, 8, 0.38, 243, 65, 335, 121
159, car, 3, 0.51, 690, 131, 751, 183
159, car, 3, 0.53, 57, 184, 160, 255
159,

202, car, 3, 0.22, 520, 29, 547, 44
202, car, 3, 0.43, 691, 132, 750, 183
202, car, 3, 0.54, 55, 184, 160, 253
202, car, 3, 0.40, 702, 179, 778, 253
202, car, 3, 0.20, 0, 213, 56, 322
202, truck, 8, 0.35, 240, 67, 333, 121
203, car, 3, 0.21, 521, 29, 547, 44
203, car, 3, 0.46, 690, 132, 750, 183
203, car, 3, 0.53, 55, 185, 160, 253
203, car, 3, 0.42, 702, 180, 778, 252
203, car, 3, 0.20, 0, 213, 55, 322
203, truck, 8, 0.36, 240, 66, 334, 122
204, car, 3, 0.43, 691, 133, 750, 183
204, car, 3, 0.54, 53, 182, 162, 257
204, car, 3, 0.43, 701, 180, 778, 252
204, car, 3, 0.27, 0, 215, 57, 320
204, truck, 8, 0.39, 239, 65, 336, 122
205, car, 3, 0.21, 520, 29, 546, 43
205, car, 3, 0.43, 690, 133, 750, 183
205, car, 3, 0.53, 54, 182, 162, 257
205, car, 3, 0.43, 701, 181, 779, 251
205, car, 3, 0.23, 0, 215, 56, 320
205, truck, 8, 0.40, 239, 65, 336, 123
206, car, 3, 0.20, 520, 29, 546, 43
206, car, 3, 0.42, 690, 133, 750, 183
206, car, 3, 0.52, 53, 182, 162, 256
206, car, 3, 0.43, 701, 180, 778,

250, car, 3, 0.38, 693, 130, 750, 185
250, car, 3, 0.50, 37, 180, 149, 253
250, truck, 8, 0.44, 241, 66, 333, 122
251, car, 3, 0.20, 514, 27, 543, 43
251, car, 3, 0.38, 691, 130, 749, 184
251, car, 3, 0.51, 40, 181, 147, 251
251, car, 3, 0.20, 703, 173, 764, 255
251, truck, 8, 0.46, 237, 64, 332, 122
252, car, 3, 0.23, 516, 25, 546, 41
252, car, 3, 0.20, 631, 27, 659, 51
252, car, 3, 0.21, 694, 69, 769, 128
252, car, 3, 0.27, 690, 130, 750, 184
252, car, 3, 0.52, 37, 180, 146, 252
252, car, 3, 0.28, 700, 175, 762, 254
252, car, 3, 0.35, 0, 172, 53, 368
252, truck, 8, 0.37, 231, 64, 333, 123
253, car, 3, 0.20, 513, 27, 545, 42
253, car, 3, 0.31, 686, 131, 746, 184
253, car, 3, 0.45, 33, 182, 145, 252
253, truck, 8, 0.36, 236, 66, 330, 124
254, car, 3, 0.22, 619, 22, 680, 68
254, car, 3, 0.55, 29, 183, 148, 257
254, car, 3, 0.20, 0, 210, 48, 326
254, truck, 8, 0.42, 236, 66, 324, 125
255, car, 3, 0.21, 512, 26, 541, 41
255, car, 3, 0.57, 21, 184, 139, 256
255, truck, 8, 0.41, 232, 68, 32